STEP 1 — تحميل الداتا + توحيد أسماء الأعمدة
🧠 علاش؟

باش ما نطيحوش فمشاكل ديال:

spaces

الحروف الكبيرة/الصغيرة

الأسماء الطويلة

In [2]:
import pandas as pd
from pathlib import Path

BASE_DIR = Path.cwd().parent
RAW_PATH = BASE_DIR / "data" / "raw" / "ride_bookings.csv"

df = pd.read_csv(RAW_PATH)


df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

df.columns

Index(['date', 'time', 'booking_id', 'booking_status', 'customer_id',
       'vehicle_type', 'pickup_location', 'drop_location', 'avg_vtat',
       'avg_ctat', 'cancelled_rides_by_customer',
       'reason_for_cancelling_by_customer', 'cancelled_rides_by_driver',
       'driver_cancellation_reason', 'incomplete_rides',
       'incomplete_rides_reason', 'booking_value', 'ride_distance',
       'driver_ratings', 'customer_rating', 'payment_method'],
      dtype='object')

في هذه الخطوة قمنا بتحميل البيانات وتوحيد أسماء الأعمدة لتسهيل التعامل معها أثناء التحليل وتفادي الأخطاء البرمجية.

🔹 STEP 2 — تنظيف التاريخ (date)
🧠 علاش؟

باش نقدر:

نعرف الشهر

نعرف اليوم

نفرّق بين week و weekend

In [3]:
df["date"] = pd.to_datetime(df["date"], errors="coerce")

df["month"] = df["date"].dt.month
df["day_name"] = df["date"].dt.day_name()
df["is_weekend"] = df["day_name"].isin(["Saturday", "Sunday"])

df[["date", "month", "day_name", "is_weekend"]].head()

,date,month,day_name,is_weekend
0,2024-03-23,3,Saturday,True
1,2024-11-29,11,Friday,False
2,2024-08-23,8,Friday,False
3,2024-10-21,10,Monday,False
4,2024-09-16,9,Monday,False


تم تحويل عمود التاريخ إلى نوع datetime وإنشاء أعمدة جديدة لتحليل النشاط حسب الأشهر والأيام وعطلة نهاية الأسبوع.

STEP 3 — تنظيف الوقت (time) واستخراج الساعة
🧠 علاش؟

باش نعرف:

ساعات الذروة

أوقات الإلغاء

In [4]:
df["hour"] = pd.to_datetime(df["time"], errors="coerce").dt.hour

df[["time", "hour"]].head(10)


C:\Users\HP\AppData\Local\Temp\ipykernel_23172\1665996893.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["hour"] = pd.to_datetime(df["time"], errors="coerce").dt.hour


,time,hour
0,12:29:38,12
1,18:01:39,18
2,08:56:10,8
3,17:17:25,17
4,22:08:00,22
5,09:44:56,9
6,15:45:58,15
7,17:37:37,17
8,12:49:09,12
9,19:06:48,19


ملاحظة

تم استخراج الساعة من وقت الرحلة لتحليل عدد الحجوزات حسب ساعات اليوم.



STEP 4 — تنظيف الأعمدة النصية الأساسية
 علاش؟

باش:

القيم تكون موحّدة

filters ديال dashboard يخدمو مزيان

In [5]:
df["booking_status"] = df["booking_status"].astype(str).str.strip().str.upper()
df["vehicle_type"] = df["vehicle_type"].astype(str).str.strip().str.upper()
df["payment_method"] = df["payment_method"].astype(str).str.strip().str.upper()

df[["booking_status", "vehicle_type", "payment_method"]].head()


,booking_status,vehicle_type,payment_method
0,NO DRIVER FOUND,EBIKE,NAN
1,INCOMPLETE,GO SEDAN,UPI
2,COMPLETED,AUTO,DEBIT CARD
3,COMPLETED,PREMIER SEDAN,UPI
4,COMPLETED,BIKE,UPI


ملاحظة

تم توحيد القيم النصية للأعمدة الأساسية لتفادي التكرار وضمان دقة التحليل

STEP 5 — تنظيف أعمدة الأسباب (reasons)
علاش؟

باش:

نسهّلو التحليل

ما نبقاوش نتعاملو مع NaN فـ الرسوم

In [6]:
df["reason_for_cancelling_by_customer"] = (
    df["reason_for_cancelling_by_customer"]
    .fillna("NO_ISSUE")
    .str.strip()
    .str.upper()
)

df["driver_cancellation_reason"] = (
    df["driver_cancellation_reason"]
    .fillna("NO_ISSUE")
    .str.strip()
    .str.upper()
)

df["incomplete_rides_reason"] = (
    df["incomplete_rides_reason"]
    .fillna("NO_ISSUE")
    .str.strip()
    .str.upper()
)

df[
    [
        "reason_for_cancelling_by_customer",
        "driver_cancellation_reason",
        "incomplete_rides_reason",
    ]
].head(10)


,reason_for_cancelling_by_customer,driver_cancellation_reason,incomplete_rides_reason
0,NO_ISSUE,NO_ISSUE,NO_ISSUE
1,NO_ISSUE,NO_ISSUE,VEHICLE BREAKDOWN
2,NO_ISSUE,NO_ISSUE,NO_ISSUE
3,NO_ISSUE,NO_ISSUE,NO_ISSUE
4,NO_ISSUE,NO_ISSUE,NO_ISSUE
5,NO_ISSUE,NO_ISSUE,NO_ISSUE
6,NO_ISSUE,NO_ISSUE,NO_ISSUE
7,NO_ISSUE,NO_ISSUE,NO_ISSUE
8,NO_ISSUE,NO_ISSUE,NO_ISSUE
9,NO_ISSUE,NO_ISSUE,OTHER ISSUE


ملاحظة

تم تعويض القيم المفقودة في أعمدة الأسباب بقيمة NO_ISSUE لتسهيل التحليل والتمثيل البياني.

STEP 6 — حفظ البيانات المنظفة
علاش؟

باش:

نستخدمها فـ EDA

نقراها فـ Dashboard

In [7]:
PROCESSED_PATH = BASE_DIR / "data" / "processed" / "ride_bookings_clean.csv"
PROCESSED_PATH.parent.mkdir(parents=True, exist_ok=True)

df.to_csv(PROCESSED_PATH, index=False)
print("✅ تم حفظ البيانات المنظفة بنجاح")


✅ تم حفظ البيانات المنظفة بنجاح


ملاحظة

تم حفظ البيانات المنظفة في ملف جديد لاستخدامها في التحليل وبناء لوحة التحكم